# Save gradient images

In [1]:
# Import modules and R
import numpy as np, pandas as pd
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

In [2]:
# Import local code
import sys
sys.path.append('../code')
from gradientVersion import *
from processing_helpers import *
from analysis_helpers import *
from triplets import *
from mri_maps import *
from enrichments import *
from brainspan import *
%R source("../code/Rplots.R");
%R source("../code/brainPlots.R");
%R source("../code/enrichmentsPlots.R");
%R source("../code/plot_maps.R");
%R source("../code/brainspanPlots.R");

/opt/conda/lib/python3.8/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [3]:
dm_hcp_3d_ds5 = gradientVersion(approach='dm', kernel='normalized_angle', alpha=1).fit('hcp_3donors_ds5')
dm_hcp_ds9 = gradientVersion(approach='dm', kernel='normalized_angle', alpha=1).fit('hcp_ds9')

New gradients version: method=dm, kernel=normalized_angle, sparsity=0, data=hcp_3donors_ds5
New gradients version: method=dm, kernel=normalized_angle, sparsity=0, data=hcp_ds9


In [17]:
g1 = dm_hcp_ds9.scores[0].reindex(range(1,181)).values

Put gradients into vertices

In [28]:
from nilearn.input_data import NiftiLabelsMasker
from neuromaps.nulls.spins import parcels_to_vertices
from neuromaps.images import annot_to_gifti, construct_shape_gii

In [15]:
hcp_img_mni = nib.load("../data/parcellations/HCPMMP1_on_MNI152_ICBM2009a_nlin.nii.gz")
hcp_img = annot_to_gifti("../data/parcellations/lh.HCPMMP1.annot")
nib.save(hcp_img[0], '../data/parcellations/fsaverage.HCPMMP1.gii')

In [35]:
g_nifti

NiftiLabelsMasker(labels_img=<nibabel.nifti1.Nifti1Image object at 0x2b72c66558b0>)

In [36]:
version = dm_hcp_ds9
for g in range(3):
    g_scores = version.scores[g].reindex(range(1,181)).values
    g_array = parcels_to_vertices(g_scores, hcp_img)
    g_gifti = construct_shape_gii(g_array)
    nib.save(g_gifti, f"../outputs/g{g+1}.gii")
    
    g_2d = np.atleast_2d(g_scores)
    g_nifti = NiftiLabelsMasker(hcp_img_mni)
    g_nifti = g_nifti.fit().inverse_transform(g_2d)
    nib.save(g_nifti, f"../outputs/g{g+1}.nii")

In [38]:
g_nifti